multivariable 선형 모델  
데이터(8:2) 분리
1. sqft_living에 따른 price?
2. sqft_living, bedrooms, bathrooms에 따른 price?
3. sqft_living, bedrooms, bathrooms, sqft_lot, floor2, zipcode에 따른 price?

In [166]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [167]:
house_data=pd.read_csv("data/house_prices/train.csv")

In [184]:
house_data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2,2,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
3,3,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
4,4,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711


In [185]:
#1.sqft_living에 따른 price?
sqft_living=house_data["sqft_living"]
price=house_data["price"]
sqft_living=np.array(sqft_living,dtype=np.float32).reshape(-1,1)
price=np.array(price,dtype=np.float32).reshape(-1,1)
print(np.shape(sqft_living))
print(np.shape(price))

(15035, 1)
(15035, 1)


In [186]:
# train, test 데이터 분리
from sklearn.model_selection import train_test_split
sqft_living_train,sqft_living_test,price_train,price_test=train_test_split(sqft_living,
                                             price, test_size=0.2, random_state=42)
np.shape(price_train)

(12028, 1)

In [187]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(sqft_living_train)
sqft_train_scaled=scaler.transform(sqft_living_train)
sqft_test_scaled=scaler.transform(sqft_living_test)
np.shape(sqft_train_scaled)

(12028, 1)

In [188]:
x=tf.placeholder(tf.float32, shape=[None,1])
y=tf.placeholder(tf.float32, shape=[None,1])
w=tf.Variable(tf.random_normal([1]), name="weight")
b=tf.Variable(tf.random_normal([1]), name="bias")

In [189]:
hf=x*w+b
cost=tf.reduce_mean(tf.square(hf-y))
train=tf.train.GradientDescentOptimizer(0.001).minimize(cost)

In [190]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(30001):
    hv, cv, _ = sess.run([hf, cost, train], feed_dict={x:sqft_train_scaled, y:price_train})
    if step % 3000 == 0:
        print(step, "cost:", cv, "prediction:", hv)

0 cost: 432395780000.0 prediction: [[0.11502279]
 [0.11629273]
 [0.21661797]
 ...
 [0.25281125]
 [0.1277222 ]
 [0.15756577]]
3000 cost: 133910920000.0 prediction: [[527490.1 ]
 [527757.94]
 [548916.56]
 ...
 [556549.75]
 [530168.44]
 [536462.44]]
6000 cost: 130371240000.0 prediction: [[520487.9 ]
 [520910.06]
 [554260.75]
 ...
 [566292.4 ]
 [524709.5 ]
 [534630.3 ]]
9000 cost: 127030910000.0 prediction: [[512531.47]
 [513103.34]
 [558281.56]
 ...
 [574580.06]
 [518250.22]
 [531689.3 ]]
12000 cost: 123877425000.0 prediction: [[504800.38]
 [505517.72]
 [562188.  ]
 ...
 [582632.3 ]
 [511973.8 ]
 [528831.44]]
15000 cost: 120900310000.0 prediction: [[497288.8]
 [498147.5]
 [565983.8]
 ...
 [590456.4]
 [505875.7]
 [526054.9]]
18000 cost: 118089700000.0 prediction: [[489989.78]
 [490985.78]
 [569671.4 ]
 ...
 [598057.94]
 [499949.97]
 [523356.44]]
21000 cost: 115436330000.0 prediction: [[482895.88]
 [484025.3 ]
 [573252.3 ]
 ...
 [605441.8 ]
 [494190.44]
 [520732.62]]
24000 cost: 11293131600

In [183]:
error_list=[]
for i,data in enumerate(sqft_living_test):
    hv=sess.run(hf,feed_dict={x:data.reshape(-1,1)})
    print("예측값 : %f ,  실제값 : %f ,  오차 : %f" % (hv, price_test[i], hv-price_test[i]))
    error_list.append(abs(hv-price_test[i]))
median=np.median(error_list)
print("중앙값 : %f" % median)

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_26' with dtype float and shape [?,3]
	 [[node Placeholder_26 (defined at <ipython-input-179-90266fe5ddc0>:1) ]]

Original stack trace for 'Placeholder_26':
  File "C:\Users\user\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\user\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\user\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\user\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\Users\user\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\user\Anaconda3\lib\asyncio\base_even

In [193]:
#2. sqft_living, bedrooms, bathrooms에 따른 price?
xdata=house_data[["sqft_living","bedrooms","bathrooms"]]
xdata=np.array(xdata, dtype=np.float32)
ydata=price
print(np.shape(xdata),np.shape(ydata))

(15035, 3) (15035, 1)


In [194]:
# train, test 데이터 분리
from sklearn.model_selection import train_test_split
xdata_train,xdata_test,ydata_train,ydata_test=train_test_split(xdata,
                                             ydata, test_size=0.2, random_state=42)
print(np.shape(xdata_train),np.shape(ydata_train))

(12028, 3) (12028, 1)


In [195]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(xdata_train)
x_train_scale=scaler.transform(xdata_train)
x_test_scale=scaler.transform(xdata_test)
np.shape(x_train_scale)

(12028, 3)

In [196]:
x1=tf.placeholder(tf.float32, shape=[None,3])
y1=tf.placeholder(tf.float32, shape=[None,1])
w1=tf.Variable(tf.random_normal([3,1]), name="weight")
b1=tf.Variable(tf.random_normal([1]), name="bias")

In [197]:
hf=tf.matmul(x1,w1)+b1
cost=tf.reduce_mean(tf.square(hf-y1))
train=tf.train.GradientDescentOptimizer(0.001).minimize(cost)
sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [198]:
for step in range(30001):
    hv, cv, _ = sess.run([hf, cost, train], feed_dict={x1:x_train_scale, y1:ydata_train})
    if step % 3000 == 0:
        print(step, "cost:", cv, "prediction:", hv)

0 cost: 432394080000.0 prediction: [[1.4744895]
 [1.5768094]
 [1.7613589]
 ...
 [1.8907278]
 [1.5024211]
 [1.6744136]]
3000 cost: 124293200000.0 prediction: [[480437.53]
 [519468.72]
 [556451.9 ]
 ...
 [582518.4 ]
 [482777.12]
 [546021.9 ]]
6000 cost: 118851160000.0 prediction: [[458301.2 ]
 [512039.47]
 [558105.7 ]
 ...
 [595196.9 ]
 [461951.9 ]
 [545860.4 ]]
9000 cost: 114186445000.0 prediction: [[437707.22]
 [504908.44]
 [559254.75]
 ...
 [606540.9 ]
 [442594.9 ]
 [545338.3 ]]
12000 cost: 110179660000.0 prediction: [[418863.38]
 [498397.53]
 [560309.25]
 ...
 [617035.3 ]
 [424920.2 ]
 [544845.75]]
15000 cost: 106730140000.0 prediction: [[401625.7]
 [492452.4]
 [561278.1]
 ...
 [626748.5]
 [408788.9]
 [544378.6]]
18000 cost: 103752810000.0 prediction: [[385863.06]
 [487024.44]
 [562170.7 ]
 ...
 [635744.25]
 [394074.88]
 [543934.4 ]]
21000 cost: 101175820000.0 prediction: [[371454.44]
 [482068.44]
 [562994.7 ]
 ...
 [644080.75]
 [380661.25]
 [543510.1 ]]
24000 cost: 98938520000.0 pre

In [199]:
error_list1=[]
for i,data in enumerate(x_test_scale):
    hv=sess.run(hf,feed_dict={x1:data.reshape(-1,3)})
    print("예측값 : %f ,  실제값 : %f ,  오차 : %f" % (hv,ydata_test[i], hv-ydata_test[i]))
    error_list1.append(abs(hv-ydata_test[i]))
# print(np.mean(error_list))
median1=np.median(error_list1)
print("중앙값 :", median1)

예측값 : 529228.875000 ,  실제값 : 256000.000000 ,  오차 : 273228.875000
예측값 : 500647.187500 ,  실제값 : 235000.000000 ,  오차 : 265647.187500
예측값 : 436286.687500 ,  실제값 : 626700.000000 ,  오차 : -190413.312500
예측값 : 320988.875000 ,  실제값 : 245500.000000 ,  오차 : 75488.875000
예측값 : 579732.875000 ,  실제값 : 593450.000000 ,  오차 : -13717.125000
예측값 : 464435.093750 ,  실제값 : 400000.000000 ,  오차 : 64435.093750
예측값 : 512990.406250 ,  실제값 : 320000.000000 ,  오차 : 192990.406250
예측값 : 577946.375000 ,  실제값 : 465000.000000 ,  오차 : 112946.375000
예측값 : 488255.031250 ,  실제값 : 323000.000000 ,  오차 : 165255.031250
예측값 : 564363.250000 ,  실제값 : 250000.000000 ,  오차 : 314363.250000
예측값 : 586396.687500 ,  실제값 : 925000.000000 ,  오차 : -338603.312500
예측값 : 317964.718750 ,  실제값 : 575000.000000 ,  오차 : -257035.281250
예측값 : 611065.375000 ,  실제값 : 425000.000000 ,  오차 : 186065.375000
예측값 : 558570.500000 ,  실제값 : 770000.000000 ,  오차 : -211429.500000
예측값 : 405642.968750 ,  실제값 : 200000.000000 ,  오차 : 205642.968750
예측값 : 548284.750000 ,  

예측값 : 576868.750000 ,  실제값 : 520000.000000 ,  오차 : 56868.750000
예측값 : 539352.312500 ,  실제값 : 438000.000000 ,  오차 : 101352.312500
예측값 : 553644.250000 ,  실제값 : 310000.000000 ,  오차 : 243644.250000
예측값 : 536537.125000 ,  실제값 : 560000.000000 ,  오차 : -23462.875000
예측값 : 651785.875000 ,  실제값 : 1010000.000000 ,  오차 : -358214.125000
예측값 : 542492.062500 ,  실제값 : 559000.000000 ,  오차 : -16507.937500
예측값 : 629272.625000 ,  실제값 : 579000.000000 ,  오차 : 50272.625000
예측값 : 493614.531250 ,  실제값 : 440000.000000 ,  오차 : 53614.531250
예측값 : 510930.625000 ,  실제값 : 161700.000000 ,  오차 : 349230.625000
예측값 : 403647.593750 ,  실제값 : 356000.000000 ,  오차 : 47647.593750
예측값 : 564845.375000 ,  실제값 : 272000.000000 ,  오차 : 292845.375000
예측값 : 650161.687500 ,  실제값 : 560000.000000 ,  오차 : 90161.687500
예측값 : 412580.062500 ,  실제값 : 156601.000000 ,  오차 : 255979.062500
예측값 : 273302.343750 ,  실제값 : 90000.000000 ,  오차 : 183302.343750
예측값 : 562076.875000 ,  실제값 : 230000.000000 ,  오차 : 332076.875000
예측값 : 466221.593750 ,  실제값 : 

예측값 : 545147.312500 ,  실제값 : 720000.000000 ,  오차 : -174852.687500
예측값 : 414962.062500 ,  실제값 : 349000.000000 ,  오차 : 65962.062500
예측값 : 367757.687500 ,  실제값 : 334500.000000 ,  오차 : 33257.687500
예측값 : 365102.437500 ,  실제값 : 299000.000000 ,  오차 : 66102.437500
예측값 : 519863.093750 ,  실제값 : 375000.000000 ,  오차 : 144863.093750
예측값 : 642533.500000 ,  실제값 : 745000.000000 ,  오차 : -102466.500000
예측값 : 599544.312500 ,  실제값 : 300000.000000 ,  오차 : 299544.312500
예측값 : 592993.812500 ,  실제값 : 405000.000000 ,  오차 : 187993.812500
예측값 : 506166.656250 ,  실제값 : 402000.000000 ,  오차 : 104166.656250
예측값 : 520731.875000 ,  실제값 : 482000.000000 ,  오차 : 38731.875000
예측값 : 379989.875000 ,  실제값 : 640000.000000 ,  오차 : -260010.125000
예측값 : 585687.875000 ,  실제값 : 449000.000000 ,  오차 : 136687.875000
예측값 : 361482.781250 ,  실제값 : 400000.000000 ,  오차 : -38517.218750
예측값 : 697572.625000 ,  실제값 : 411500.000000 ,  오차 : 286072.625000
예측값 : 573777.875000 ,  실제값 : 579000.000000 ,  오차 : -5222.125000
예측값 : 572264.687500 ,  실제값 

예측값 : 778833.750000 ,  실제값 : 603000.000000 ,  오차 : 175833.750000
예측값 : 556897.375000 ,  실제값 : 575000.000000 ,  오차 : -18102.625000
예측값 : 689464.625000 ,  실제값 : 875000.000000 ,  오차 : -185535.375000
예측값 : 544711.750000 ,  실제값 : 525000.000000 ,  오차 : 19711.750000
예측값 : 856248.625000 ,  실제값 : 1222500.000000 ,  오차 : -366251.375000
예측값 : 568967.250000 ,  실제값 : 825000.000000 ,  오차 : -256032.750000
예측값 : 583419.187500 ,  실제값 : 1350000.000000 ,  오차 : -766580.812500
예측값 : 752631.750000 ,  실제값 : 1125000.000000 ,  오차 : -372368.250000
예측값 : 649566.187500 ,  실제값 : 749000.000000 ,  오차 : -99433.812500
예측값 : 379394.406250 ,  실제값 : 225000.000000 ,  오차 : 154394.406250
예측값 : 650001.750000 ,  실제값 : 1140000.000000 ,  오차 : -489998.250000
예측값 : 319797.875000 ,  실제값 : 407500.000000 ,  오차 : -87702.125000
예측값 : 528037.875000 ,  실제값 : 499000.000000 ,  오차 : 29037.875000
예측값 : 688273.625000 ,  실제값 : 760000.000000 ,  오차 : -71726.375000
예측값 : 611614.250000 ,  실제값 : 399950.000000 ,  오차 : 211664.250000
예측값 : 314920.5625

예측값 : 601284.125000 ,  실제값 : 925000.000000 ,  오차 : -323715.875000
예측값 : 498491.812500 ,  실제값 : 849900.000000 ,  오차 : -351408.187500
예측값 : 824091.625000 ,  실제값 : 1965220.000000 ,  오차 : -1141128.375000
예측값 : 512830.437500 ,  실제값 : 361000.000000 ,  오차 : 151830.437500
예측값 : 440455.187500 ,  실제값 : 319000.000000 ,  오차 : 121455.187500
예측값 : 462648.593750 ,  실제값 : 245000.000000 ,  오차 : 217648.593750
예측값 : 299826.500000 ,  실제값 : 273000.000000 ,  오차 : 26826.500000
예측값 : 458480.093750 ,  실제값 : 274800.000000 ,  오차 : 183680.093750
예측값 : 611889.750000 ,  실제값 : 437850.000000 ,  오차 : 174039.750000
예측값 : 585092.375000 ,  실제값 : 495000.000000 ,  오차 : 90092.375000
예측값 : 516838.968750 ,  실제값 : 396500.000000 ,  오차 : 120338.968750
예측값 : 343022.312500 ,  실제값 : 175000.000000 ,  오차 : 168022.312500
예측값 : 410198.062500 ,  실제값 : 459000.000000 ,  오차 : -48801.937500
예측값 : 619951.250000 ,  실제값 : 310000.000000 ,  오차 : 309951.250000
예측값 : 479918.031250 ,  실제값 : 712500.000000 ,  오차 : -232581.968750
예측값 : 341831.312500 ,

예측값 : 365102.437500 ,  실제값 : 405000.000000 ,  오차 : -39897.562500
예측값 : 511959.375000 ,  실제값 : 469000.000000 ,  오차 : 42959.375000
예측값 : 591207.375000 ,  실제값 : 385000.000000 ,  오차 : 206207.375000
예측값 : 494852.156250 ,  실제값 : 362500.000000 ,  오차 : 132352.156250
예측값 : 440132.968750 ,  실제값 : 412250.000000 ,  오차 : 27882.968750
예측값 : 681447.500000 ,  실제값 : 850000.000000 ,  오차 : -168552.500000
예측값 : 591320.625000 ,  실제값 : 310000.000000 ,  오차 : 281320.625000
예측값 : 602361.875000 ,  실제값 : 800000.000000 ,  오차 : -197638.125000
예측값 : 622768.750000 ,  실제값 : 495000.000000 ,  오차 : 127768.750000
예측값 : 466108.281250 ,  실제값 : 474905.000000 ,  오차 : -8796.718750
예측값 : 538596.875000 ,  실제값 : 530000.000000 ,  오차 : 8596.875000
예측값 : 463244.093750 ,  실제값 : 565000.000000 ,  오차 : -101755.906250
예측값 : 671004.125000 ,  실제값 : 715000.000000 ,  오차 : -43995.875000
예측값 : 898413.375000 ,  실제값 : 1830000.000000 ,  오차 : -931586.625000
예측값 : 867883.000000 ,  실제값 : 2300000.000000 ,  오차 : -1432117.000000
예측값 : 559003.750000 , 

예측값 : 806226.625000 ,  실제값 : 1700000.000000 ,  오차 : -893773.375000
예측값 : 436995.500000 ,  실제값 : 439000.000000 ,  오차 : -2004.500000
예측값 : 418421.718750 ,  실제값 : 416000.000000 ,  오차 : 2421.718750
예측값 : 451929.593750 ,  실제값 : 445950.000000 ,  오차 : 5979.593750
예측값 : 500484.906250 ,  실제값 : 305000.000000 ,  오차 : 195484.906250
예측값 : 646106.500000 ,  실제값 : 1000750.000000 ,  오차 : -354643.500000
예측값 : 371057.406250 ,  실제값 : 330000.000000 ,  오차 : 41057.406250
예측값 : 735042.375000 ,  실제값 : 495000.000000 ,  오차 : 240042.375000
예측값 : 693633.125000 ,  실제값 : 921500.000000 ,  오차 : -227866.875000
예측값 : 448905.468750 ,  실제값 : 345000.000000 ,  오차 : 103905.468750
예측값 : 640038.250000 ,  실제값 : 650000.000000 ,  오차 : -9961.750000
예측값 : 784193.250000 ,  실제값 : 1288000.000000 ,  오차 : -503806.750000
예측값 : 721874.750000 ,  실제값 : 583000.000000 ,  오차 : 138874.750000
예측값 : 613240.750000 ,  실제값 : 316000.000000 ,  오차 : 297240.750000
예측값 : 616218.250000 ,  실제값 : 834000.000000 ,  오차 : -217781.750000
예측값 : 488255.031250 ,  실

In [147]:
np.shape(x_test_scale)

(3007, 3)

In [148]:
np.shape(ydata_test)

(3007, 1)